In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from yellowbrick.classifier import ConfusionMatrix

In [2]:
stock_data=pd.read_pickle("process_data/clean_data/sandp_ta.plk")

In [3]:
os.chdir(r"C:\Users\power\Dropbox\Cory")

In [ ]:
quarterly_fin=pd.read_csv("SP500_quarterly_fin.zip")

In [ ]:
from IPython.display import display, HTML
pd.options.display.max_columns=None

In [ ]:
date_vars=['datadate','fyearq','fqtr','fyr']

num_vars=['acchgq','acoq','actq','altoq','ancq','anoq',\
'aociderglq','aociotherq','aocisecglq','aol2q','aoq','apq','aqaq',\
'aqdq','aqepsq','aqpl1q','aqpq','arcedq','arceepsq','arceq','atq',\
'aul3q','billexceq','capr1q','capr2q','capsftq','capsq','capxy',\
'cdvcy','chechy','ceiexbillq','ceqq','cheq','chq','cibegniq','cicurrq',\
'ciderglq','cimiiq','ciotherq','ciy','cogsq','cshfd12','cshfdq','cshiq',\
'cshopq','cshoq','cshprq','cshtrq','cstkeq','cstkq','dcomq','dd1q','deracq',\
'deraltq','derhedglq','derlcq','derlltq','diladq','dlcq','dlttq','doq',\
'dpacreq','dpactq','dpq','dpretq','drcq','drltq','dteaq','dtedq','dteepsq',\
'dtepq','dvintfq','dvpq','dvpspq','epsfiq','epsfxy','esoptq','esubq','fcaq',\
'ffoq','finacoq','finchq','findlcq','findltq','finivstq','finltoq','finreccq',\
'finrecltq','finrevq','finxintq','finxoprq','gdwlamq','gdwliaq','gdwlidq',\
'gdwliepsq','gdwlipq','gdwlq','glaq','glceaq','glceepsq','glcepq','glivq',\
'glpq', 'hedgeglq','ibq','icaptq','intaccq','intanoq','intanq','invfgq','invtq',\
'itccy','ivaeqq','ivltq','ivstq','lctq','lltq','lnoq','lol2q','lul3q','ltq','mibnq',\
'mibq','mibtq','miiq','mkvaltq','msaq','ncoq','niitq','nimq','niq','nopiq','npatq',\
'npq','nrtxtepsq','nrtxtq','obkq','oepsxq','oiadpq','opepsq','piq','pllq',\
'pncepsq','pncpq','pstkq','rcaq','rcdq','recdq','recdq','rdipq','rectq','req',\
'revtq','rllq','rraq','saleq','setaq','spceq','spiq','tiiq','tieq','txdbaq','txdbclq',\
'txdbq','txwq','uaptq','uceqq','wcapq','wdaq','xaccq','xiq']

cat_vars=['gvkey','tic','cusip','idbflag','loc','naics','state']

In [ ]:
quarterly_info=quarterly_fin[num_vars].describe().T

In [ ]:
num_filtered=list(quarterly_info[quarterly_info['count']>13000].index)

In [ ]:
quarterly_fin_filtered=quarterly_fin[date_vars+num_filtered+cat_vars]

In [ ]:
quarterly_fin_filtered.loc[quarterly_fin_filtered['state'].isnull(),'state']=quarterly_fin_filtered['loc']

In [ ]:
quarterly_fin_filtered.fillna(0, inplace=True)

In [ ]:
quarterly_fin_filtered['year']=np.floor(quarterly_fin_filtered['datadate']/10000)
quarterly_fin_filtered['quarter']=np.floor((quarterly_fin_filtered['datadate']-quarterly_fin_filtered['year']*10000)/100)//3+quarterly_fin_filtered['year']*100

In [ ]:
states=pd.get_dummies(quarterly_fin_filtered['state'])
states.columns=["state_"+str(i) for i in states.columns]

In [ ]:
naics=pd.get_dummies(quarterly_fin_filtered['naics'])
naics.columns=["naics_"+str(i) for i in naics.columns]

In [ ]:
idbflag=pd.get_dummies(quarterly_fin_filtered['idbflag'])
idbflag.columns=["idbflag_"+str(i) for i in idbflag.columns]

In [ ]:
quarterly_fin_filtered=pd.concat([quarterly_fin_filtered, states, naics,idbflag], axis=1).drop(['idbflag','loc','naics','state'], axis=1)

In [ ]:
quarterly_fin_filtered.to_pickle("quarterly_spy.pkl")

In [ ]:
institutional=pd.read_csv("stock_institutional_ownership.csv", encoding='latin1')

In [ ]:
institutional_vars=['quarter','TIC','mktcap','io_usd', 'nbr_firms', 'io', \
                    'io_dom', 'io_for', 'io_for_us','io_for_nus', 'io_dom_indep', \
                    'io_dom_grey', 'io_for_indep',\
                    'io_for_grey', 'io_cat1', 'io_cat2', 'io_cat3', 'io_cat4', 'io_cat5',\
                    'io_cat6', 'io_cat7', 'io_indep', 'io_grey', 'io_common', 'io_civil',\
                    'io_for_common', 'io_for_civil', 'ibh_5pct', 'ibh_1pct', 'top5','herf']

In [ ]:
institutional_filtered=institutional[institutional_vars]

In [ ]:
institutional_filtered.rename(columns={'TIC':'tic'}, inplace=True)

In [ ]:
institutional_filtered.drop_duplicates(['tic','quarter'], inplace=True)

In [ ]:
fin_and_inst=quarterly_fin_filtered.merge(institutional_filtered, on=['tic','quarter'])

In [ ]:
fin_and_inst.to_pickle("quarterly_spy_w_inst.pickle")

os.listdir("execucomp/")

comp_fin_dir_comp=pd.read_csv("execucomp/Company Financial and Director Compensation for 2005 and prior.zip")
def_comp=pd.read_csv("execucomp/deferred compensation.zip", encoding='latin1')

def_comp.columns

In [4]:
quarterly_fin_filtered=pd.read_pickle("quarterly_spy.pkl")

In [5]:
quarterly_fin_filtered['date']=pd.to_datetime(quarterly_fin_filtered['datadate'].astype(str),format='%Y%m%d')

In [6]:
quarterly_fin_filtered.set_index(['date'], inplace=True)

In [7]:
quarterly_fin_to_merge=quarterly_fin_filtered.groupby('tic').apply(lambda x: x.asfreq("D"))

In [8]:
merged=stock_data.drop('tic',axis=1).join(quarterly_fin_to_merge.drop('tic',axis=1), how='left')

In [20]:
merged_trim=merged.loc[:,:'xiq'].reset_index()

In [26]:
list(merged_trim.columns)

['tic',
 'date',
 'close',
 'high',
 'low',
 'open',
 '30 period CCI',
 'MACD',
 'SIGNAL',
 'RSI',
 'MFV',
 'volume',
 'nextwk_roi',
 'roi_class',
 'datadate',
 'fyearq',
 'fqtr',
 'fyr',
 'acchgq',
 'acoq',
 'actq',
 'altoq',
 'ancq',
 'anoq',
 'aociderglq',
 'aociotherq',
 'aocisecglq',
 'aol2q',
 'aoq',
 'apq',
 'aqpl1q',
 'atq',
 'aul3q',
 'capsq',
 'capxy',
 'chechy',
 'ceqq',
 'cheq',
 'chq',
 'cibegniq',
 'cicurrq',
 'ciderglq',
 'cimiiq',
 'ciotherq',
 'ciy',
 'cogsq',
 'cshfd12',
 'cshfdq',
 'cshiq',
 'cshopq',
 'cshoq',
 'cshprq',
 'cshtrq',
 'cstkeq',
 'cstkq',
 'dcomq',
 'diladq',
 'dlcq',
 'dlttq',
 'doq',
 'dpq',
 'drcq',
 'drltq',
 'dvpq',
 'dvpspq',
 'epsfiq',
 'epsfxy',
 'esoptq',
 'gdwlq',
 'ibq',
 'icaptq',
 'intanoq',
 'intanq',
 'invtq',
 'ivstq',
 'lctq',
 'lltq',
 'lnoq',
 'lol2q',
 'lul3q',
 'ltq',
 'mibnq',
 'mibq',
 'mibtq',
 'miiq',
 'mkvaltq',
 'msaq',
 'niq',
 'nopiq',
 'oepsxq',
 'oiadpq',
 'opepsq',
 'piq',
 'pstkq',
 'rdipq',
 'rectq',
 'req',
 'revtq',


In [22]:
merged_filled=merged_trim.groupby('tic', as_index=False).fillna(method='ffill')

In [24]:
merged_filled.dropna(inplace=True)

In [28]:
X = merged_filled.drop(['tic','date','close','high',\
                        'low','open','volume','nextwk_roi',\
                        'roi_class','datadate','fyearq','fqtr','fyr',], axis=1)

X_non_PCA=X.loc[:, '30 period CCI': 'MFV']
X_PCA=X.loc[:,'acchgq':]
poly = PolynomialFeatures(degree=2)
Xp = poly.fit_transform(X)

y = merged_filled.roi_class

X_train, X_test, y_train, y_test = train_test_split(Xp, y, test_size = 0.2)

sd = StandardScaler()
X_train = sd.fit_transform(X_train)
X_test = sd.fit_transform(X_test)

X_train.shape

clf = LogisticRegression(n_jobs=-1, solver='lbfgs').fit(X_train, y_train)

C:\Users\power\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [ ]:
print(clf.score(X_train, y_train))

print(clf.score(X_test, y_test))